In [2]:
import numpy as np
import pandas as pd
import sklearn
import requests
from tqdm import tqdm

# Load Data

In [3]:
username = "2wsgzl0-vwx2hipkj-yxme0ub-wgjenczp83"
password = "nQv7vWe@FtES5_Z%4@F^GbtZrA*&Rh"

matches_data = requests.get("https://apirest.wyscout.com/v3/competitions?areaId=USA", auth=(username, password))
comp_df = pd.DataFrame(matches_data.json()['competitions'])
matches_df = pd.DataFrame()
for i in tqdm(comp_df['wyId'].tolist()):
    league_data = requests.get(f"https://apirest.wyscout.com/v3/competitions/{i}/matches", auth=(username, password))
    mdf = pd.DataFrame(league_data.json()['matches'])
    matches_df = pd.concat([matches_df, mdf])
matches_df.head()

100%|██████████| 33/33 [00:20<00:00,  1.57it/s]


,matchId,label,date,dateutc,status,competitionId,seasonId,roundId,gameweek
0,5539787,"Bryant Bulldogs - Chicago State Cougars, 5-2",2023-10-20 01:00:00,2023-10-19 23:00:00,Played,43259,190032,4429616,3
1,5539502,"USC Upstate Spartans - SCSU Bulldogs, 10-0",2023-10-19 00:00:00,2023-10-18 22:00:00,Played,43259,190032,4429616,3
2,5512618,"Boston Terriers - Yale Bulldogs, 1-1",2023-10-18 01:00:00,2023-10-17 23:00:00,Played,43259,190032,4429616,3
3,5512619,"Princeton Tigers - Bucknell Bison, 0-1",2023-10-18 01:00:00,2023-10-17 23:00:00,Played,43259,190032,4429616,3
4,5512617,"Cornell Big Red - Albany Great Danes, 4-0",2023-10-17 23:00:00,2023-10-17 21:00:00,Played,43259,190032,4429616,3


# Feature Engineering on Event Data from One Match

## Load Match Event Data

In [4]:
event_data = requests.get(f"https://apirest.wyscout.com/v3/matches/{5415673}/events", auth=(username, password))
event_data = pd.DataFrame(event_data.json()['events'])
event_data.head()

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,type,location,team,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession
0,1480212789,5415673,1H,0,6,00:00:06.910,8.910437,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
1,1480212788,5415673,1H,0,6,00:00:06.914,8.914118,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
2,1480212790,5415673,1H,0,9,00:00:09.085,11.085398,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
3,1480212791,5415673,1H,0,10,00:00:10.959,12.959132,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
4,1480212792,5415673,1H,0,13,00:00:13.055,15.055166,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."


In [5]:
for index in np.arange(len(event_data)):
    row = event_data.iloc[index]
    if row['carry'] is not None:
        break
row['carry']

{'progression': 4.792, 'endLocation': {'x': 19, 'y': 63}}

## Remove Irrelevant Columns

In [6]:
event_data = event_data.drop(['minute', 'second','videoTimestamp'], axis=1)
event_data = event_data[event_data['matchPeriod'] != 'P']
event_data.head()

,id,matchId,matchPeriod,matchTimestamp,relatedEventId,type,location,team,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession
0,1480212789,5415673,1H,00:00:06.910,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
1,1480212788,5415673,1H,00:00:06.914,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
2,1480212790,5415673,1H,00:00:09.085,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
3,1480212791,5415673,1H,00:00:10.959,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."
4,1480212792,5415673,1H,00:00:13.055,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't..."


## Convert matchTimestamp to Seconds --> Add as Column

In [7]:
#Get time in seconds
match_time = []
for i in event_data['matchTimestamp'].tolist():
    time_list = i.split(':')
    seconds = float(time_list[0]) * 60 * 60
    seconds += float(time_list[1]) * 60
    seconds += float(time_list[2])
    match_time.append(seconds)
event_data['match_time_seconds'] = match_time
event_data = event_data.drop('matchTimestamp', axis=1)
event_data.head()

,id,matchId,matchPeriod,relatedEventId,type,location,team,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession,match_time_seconds
0,1480212789,5415673,1H,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",6.910
1,1480212788,5415673,1H,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",6.914
2,1480212790,5415673,1H,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",9.085
3,1480212791,5415673,1H,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",10.959
4,1480212792,5415673,1H,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",13.055


## One Hot Encode Match Period

In [8]:
first_half = []
second_half = []
extra_time_1 = []
extra_time_2 = []
for i in event_data['matchPeriod'].tolist():
    if i == '1H':
        first_half.append(1)
        second_half.append(0)
        extra_time_1.append(0)
        extra_time_2.append(0)
    elif i == '2H':
        first_half.append(0)
        second_half.append(1)
        extra_time_1.append(0)
        extra_time_2.append(0)
    elif i == '1E':
        first_half.append(0)
        second_half.append(0)
        extra_time_1.append(1)
        extra_time_2.append(0)
    else:
        first_half.append(0)
        second_half.append(0)
        extra_time_1.append(0)
        extra_time_2.append(1)
event_data = event_data.drop('matchPeriod', axis=1)
event_data['1H'] = first_half
event_data['2H'] = second_half
event_data['1E'] = extra_time_1
event_data['2E'] = extra_time_2
event_data.head()

,id,matchId,relatedEventId,type,location,team,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession,match_time_seconds,1H,2H,1E,2E
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",6.910,1,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",6.914,1,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",9.085,1,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",10.959,1,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",13.055,1,0,0,0


## Add Home Team Binary Vector

In [9]:
match_call = requests.get("https://apirest.wyscout.com/v3/matches/%s?useSides=1" % 5415673, auth=(username, password))
home_id = pd.DataFrame(match_call.json()['teamsData'])['home'].tolist()[0]
home_team = []
for i in event_data['team'].tolist():
    if i['id'] == home_id:
        home_team.append(1)
    else:
        home_team.append(0)
event_data['home_team'] = home_team
event_data.head()

,id,matchId,relatedEventId,type,location,team,opponentTeam,player,pass,shot,...,aerialDuel,infraction,carry,possession,match_time_seconds,1H,2H,1E,2E,home_team
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,...,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",6.910,1,0,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,...,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",6.914,1,0,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,...,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",9.085,1,0,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,...,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",10.959,1,0,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,...,None,None,None,"{'id': 1480212789, 'duration': '14.809237', 't...",13.055,1,0,0,0,0


## One Hot Encode Primary Action Type

In [10]:
primary_events = ['pass',
 'interception',
 'corner',
 'goalkeeper_exit',
 'shot',
 'goal_kick',
 'duel',
 'throw_in',
 'touch',
 'infraction',
 'free_kick',
 'clearance',
 'shot_against',
 'acceleration',
 'offside']
event_dict = {}
for i in primary_events:
    event_dict[i] = []
for i in event_data['type'].tolist():
    event = i['primary']
    for x in list(event_dict.keys()):
        if x == event:
            lista = event_dict[x]
            lista.append(1)
            event_dict[x] = lista
        else:
            lista = event_dict[x]
            lista.append(0)
            event_dict[x] = lista
for i in list(event_dict.keys()):
    event_data[i + '_primary_dummy'] = event_dict[i]
event_data.head()

,id,matchId,relatedEventId,type,location,team,opponentTeam,player,pass,shot,...,goal_kick_primary_dummy,duel_primary_dummy,throw_in_primary_dummy,touch_primary_dummy,infraction_primary_dummy,free_kick_primary_dummy,clearance_primary_dummy,shot_against_primary_dummy,acceleration_primary_dummy,offside_primary_dummy
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,...,0,0,0,0,0,0,0,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,...,0,0,0,0,0,0,0,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,...,0,0,0,0,0,0,0,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,...,0,0,0,0,0,0,0,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,...,0,0,0,0,0,0,0,0,0,0


## One Hot Encode Secondary Action Type

In [11]:
secondary_events = ['back_pass',
 'short_or_medium_pass',
 'lateral_pass',
 'linkup_play',
 'under_pressure',
 'forward_pass',
 'long_pass',
 'pass_to_final_third',
 'progressive_pass',
 'smart_pass',
 'through_pass',
 'cross',
 'loss',
 'pass_to_penalty_area',
 'cross_blocked',
 'recovery',
 'shot_after_corner',
 'loose_ball_duel',
 'pass',
 'counterpressing_recovery',
 'head_pass',
 'defensive_duel',
 'ground_duel',
 'dribble',
 'offensive_duel',
 'carry',
 'foul_suffered',
 'foul',
 'sliding_tackle',
 'touch_in_box',
 'deep_completion',
 'aerial_duel',
 'progressive_run',
 'dribbled_past_attempt',
 'free_kick_cross',
 'deep_completed_cross',
 'opportunity',
 'shot_assist',
 'save',
 'shot_after_throw_in',
 'shot_block',
 'hand_pass',
 'acceleration',
 'head_shot',
 'save_with_reflex',
 'interception',
 'red_card',
 'yellow_card',
 'key_pass',
'goal',
'conceded_goal',
'second_assist',
'assist',
'third_assist',
'shot_after_free_kick']
event_dict = {}
for i in secondary_events:
    event_dict[i] = []
for i in event_data['type'].tolist():
    event = i['secondary']
    if len(event) == 0:
        for x in list(event_dict.keys()):
            lista = event_dict[x]
            lista.append(0)
            event_dict[x] = lista
    else:
        for x in list(event_dict.keys()):
            if x in event:
                lista = event_dict[x]
                lista.append(1)
                event_dict[x] = lista
            else:
                lista = event_dict[x]
                lista.append(0)
                event_dict[x] = lista
for i in list(event_dict.keys()):
    event_data[i + '_secondary_dummy'] = event_dict[i]
event_data.head()

,id,matchId,relatedEventId,type,location,team,opponentTeam,player,pass,shot,...,interception_secondary_dummy,red_card_secondary_dummy,yellow_card_secondary_dummy,key_pass_secondary_dummy,goal_secondary_dummy,conceded_goal_secondary_dummy,second_assist_secondary_dummy,assist_secondary_dummy,third_assist_secondary_dummy,shot_after_free_kick_secondary_dummy
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,...,0,0,0,0,0,0,0,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,...,0,0,0,0,0,0,0,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,...,0,0,0,0,0,0,0,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,...,0,0,0,0,0,0,0,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,...,0,0,0,0,0,0,0,0,0,0


## One Hot Encode Body Part Used to Perform Action

In [12]:
foot = []
head = []
hand = []
for index, i in enumerate(event_data['head_pass_secondary_dummy'].tolist()):
    if i or event_data['head_shot_secondary_dummy'].tolist()[index]:
        head.append(1)
        foot.append(0)
        hand.append(0)
    elif (event_data['hand_pass_secondary_dummy'].tolist()[index]) or (event_data.iloc[index]['infraction'] != None and event_data.iloc[index]['infraction']['type'] == 'hand_foul'):
        hand.append(1)
        foot.append(0)
        head.append(0)
    else:
        foot.append(1)
        hand.append(0)
        head.append(0)
event_data = event_data.drop(['head_shot_secondary_dummy', 'head_pass_secondary_dummy', 'hand_pass_secondary_dummy'], axis=1)
event_data['head'] = head
event_data['foot'] = foot
event_data['hand'] = hand
event_data.head()

,id,matchId,relatedEventId,type,location,team,opponentTeam,player,pass,shot,...,key_pass_secondary_dummy,goal_secondary_dummy,conceded_goal_secondary_dummy,second_assist_secondary_dummy,assist_secondary_dummy,third_assist_secondary_dummy,shot_after_free_kick_secondary_dummy,head,foot,hand
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 29, 'y': 38}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,...,0,0,0,0,0,0,0,0,1,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 50}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,...,0,0,0,0,0,0,0,0,1,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 26, 'y': 70}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,...,0,0,0,0,0,0,0,0,1,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'x': 29, 'y': 95}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,...,0,0,0,0,0,0,0,0,1,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 24, 'y': 77}","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,...,0,0,0,0,0,0,0,0,1,0


## Get Action Start and End x,y Coordinates

In [13]:
end_x = []
end_y = []
start_x = []
start_y = []
field_length = 100
field_width = 100
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    locs = row['location'] 
    if locs != None:
        start_x.append(locs['x'])
        start_y.append(locs['y'])
    else:
        start_x.append('x')
        start_y.append('y')
        end_x.append('x')
        end_y.append('y')
        continue
    if row['player']['id'] == 0:
        end_x.append('x')
        end_y.append('y')
    elif row['pass'] != None:
        locs = row['pass']['endLocation']
        end_x.append(locs['x'])
        end_y.append(locs['y'])
    elif row['shot'] != None:
        locs = row['possession']
        if locs['eventsNumber'] == locs['eventIndex'] + 1:
            locs = locs['endLocation']
            end_x.append(locs['x'])
            end_y.append(locs['y'])
        else:
            print('ERROR')
    elif row['carry'] != None:
        locs = row['carry']['endLocation']
        end_x.append(locs['x'])
        end_y.append(locs['y'])
    else:
        locs = row['location']
        end_x.append(locs['x'])
        end_y.append(locs['y'])
event_data['end_x'] = end_x
event_data['end_y'] = end_y
event_data['start_x'] = start_x
event_data['start_y'] = start_y
event_data = event_data.drop('location', axis=1)
event_data = event_data[event_data['start_x'] != 'x']
event_data = event_data[event_data['end_x'] != 'x']
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,assist_secondary_dummy,third_assist_secondary_dummy,shot_after_free_kick_secondary_dummy,head,foot,hand,end_x,end_y,start_x,start_y
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,0,0,1,0,50,50,29,38
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,0,0,1,0,26,70,50,50
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,0,0,1,0,29,95,26,70
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,0,0,1,0,24,77,29,95
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,0,0,1,0,10,59,24,77


## Euclidean Distances

In [14]:
def euclideanDistance(start_x, start_y, end_x, end_y):
    return ((start_x - end_x)**2 + (start_y - end_y)**2)**0.5

### Calculate Euclidean Distance Between Action Start and End Location

In [15]:
start_end_distance = []
for index, i in enumerate(event_data['start_x'].tolist()):
    start_x = i
    start_y = event_data['start_y'].tolist()[index]
    end_x = event_data['end_x'].tolist()[index]
    end_y = event_data['end_y'].tolist()[index]
    euclidean_distance = euclideanDistance(start_x, start_y, end_x, end_y)
    start_end_distance.append(euclidean_distance)
event_data['start_end_distance'] = start_end_distance
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,third_assist_secondary_dummy,shot_after_free_kick_secondary_dummy,head,foot,hand,end_x,end_y,start_x,start_y,start_end_distance
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,0,1,0,50,50,29,38,24.186773
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,0,1,0,26,70,50,50,31.240999
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,0,1,0,29,95,26,70,25.179357
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,0,1,0,24,77,29,95,18.681542
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,0,1,0,10,59,24,77,22.803509


### Calculate Euclidean Distance Between Action Start and Opponent Goal

In [16]:
start_goal_distance = []
for index, i in enumerate(event_data['start_x'].tolist()):
    start_x = i
    start_y = event_data['start_y'].tolist()[index]
    end_x = 100
    end_y = 50
    euclidean_distance = euclideanDistance(start_x, start_y, end_x, end_y)
    start_goal_distance.append(euclidean_distance)
event_data['start_goal_distance_opponent'] = start_goal_distance
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,shot_after_free_kick_secondary_dummy,head,foot,hand,end_x,end_y,start_x,start_y,start_end_distance,start_goal_distance_opponent
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,1,0,50,50,29,38,24.186773,72.006944
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,1,0,26,70,50,50,31.240999,50.000000
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,1,0,29,95,26,70,25.179357,76.655072
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,1,0,24,77,29,95,18.681542,84.059503
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,1,0,10,59,24,77,22.803509,80.653580


### Calculate Euclidean Distance Between Action Start and Own Goal

In [17]:
start_goal_distance = []
for index, i in enumerate(event_data['start_x'].tolist()):
    start_x = i
    start_y = event_data['start_y'].tolist()[index]
    end_x = 0
    end_y = 50
    euclidean_distance = euclideanDistance(start_x, start_y, end_x, end_y)
    start_goal_distance.append(euclidean_distance)
event_data['start_goal_distance_own'] = start_goal_distance
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,head,foot,hand,end_x,end_y,start_x,start_y,start_end_distance,start_goal_distance_opponent,start_goal_distance_own
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,1,0,50,50,29,38,24.186773,72.006944,31.384710
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,1,0,26,70,50,50,31.240999,50.000000,50.000000
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,1,0,29,95,26,70,25.179357,76.655072,32.802439
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,1,0,24,77,29,95,18.681542,84.059503,53.535035
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,1,0,10,59,24,77,22.803509,80.653580,36.124784


### Calculate Euclidean Distance Between Action End and Opponent Goal

In [18]:
start_goal_distance = []
for index, i in enumerate(event_data['end_x'].tolist()):
    start_x = i
    start_y = event_data['end_y'].tolist()[index]
    end_x = 100
    end_y = 50
    euclidean_distance = euclideanDistance(start_x, start_y, end_x, end_y)
    start_goal_distance.append(euclidean_distance)
event_data['end_goal_distance_opponent'] = start_goal_distance
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,foot,hand,end_x,end_y,start_x,start_y,start_end_distance,start_goal_distance_opponent,start_goal_distance_own,end_goal_distance_opponent
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,1,0,50,50,29,38,24.186773,72.006944,31.384710,50.000000
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,1,0,26,70,50,50,31.240999,50.000000,50.000000,76.655072
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,1,0,29,95,26,70,25.179357,76.655072,32.802439,84.059503
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,1,0,24,77,29,95,18.681542,84.059503,53.535035,80.653580
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,1,0,10,59,24,77,22.803509,80.653580,36.124784,90.448881


### Calculate Euclidean Distance Between Action End and Own Goal

In [19]:
start_goal_distance = []
for index, i in enumerate(event_data['end_x'].tolist()):
    start_x = i
    start_y = event_data['end_y'].tolist()[index]
    end_x = 0
    end_y = 50
    euclidean_distance = euclideanDistance(start_x, start_y, end_x, end_y)
    start_goal_distance.append(euclidean_distance)
event_data['end_goal_distance_own'] = start_goal_distance
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,hand,end_x,end_y,start_x,start_y,start_end_distance,start_goal_distance_opponent,start_goal_distance_own,end_goal_distance_opponent,end_goal_distance_own
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,50,50,29,38,24.186773,72.006944,31.384710,50.000000,50.000000
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,26,70,50,50,31.240999,50.000000,50.000000,76.655072,32.802439
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,29,95,26,70,25.179357,76.655072,32.802439,84.059503,53.535035
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,24,77,29,95,18.681542,84.059503,53.535035,80.653580,36.124784
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,10,59,24,77,22.803509,80.653580,36.124784,90.448881,13.453624


## Calculate Number of Goals Each Team Has Scored and Goal Differential

In [20]:
home_goals = []
away_goals = []
delta = []
hg = 0
ag = 0
for index,i in enumerate(event_data['type'].tolist()):
    if 'goal' in i['secondary']:
        if event_data['home_team'].tolist()[index]:
            hg += 1
        else:
            ag += 1
    home_goals.append(hg)
    away_goals.append(ag)
    delta.append(hg-ag)
event_data['home_goals'] = home_goals
event_data['away_goals'] = away_goals
event_data['goal_delta'] = delta
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,start_x,start_y,start_end_distance,start_goal_distance_opponent,start_goal_distance_own,end_goal_distance_opponent,end_goal_distance_own,home_goals,away_goals,goal_delta
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,29,38,24.186773,72.006944,31.384710,50.000000,50.000000,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,50,50,31.240999,50.000000,50.000000,76.655072,32.802439,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,26,70,25.179357,76.655072,32.802439,84.059503,53.535035,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,29,95,18.681542,84.059503,53.535035,80.653580,36.124784,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,24,77,22.803509,80.653580,36.124784,90.448881,13.453624,0,0,0


## Calculate Number of Red Cards Each Team Has and Red Card Differential

In [21]:
home_reds = []
away_reds = []
delta = []
hr = 0
ar = 0
for index,i in enumerate(event_data['type'].tolist()):
    if 'red_card' in i['secondary']:
        if event_data['home_team'].tolist()[index]:
            hr += 1
        else:
            ar += 1
    home_reds.append(hr)
    away_reds.append(ar)
    delta.append(hr-ar)
event_data['home_red_cards'] = home_reds
event_data['away_red_cards'] = away_reds
event_data['red_card_delta'] = delta
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,start_goal_distance_opponent,start_goal_distance_own,end_goal_distance_opponent,end_goal_distance_own,home_goals,away_goals,goal_delta,home_red_cards,away_red_cards,red_card_delta
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,72.006944,31.384710,50.000000,50.000000,0,0,0,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,50.000000,50.000000,76.655072,32.802439,0,0,0,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,76.655072,32.802439,84.059503,53.535035,0,0,0,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,84.059503,53.535035,80.653580,36.124784,0,0,0,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,80.653580,36.124784,90.448881,13.453624,0,0,0,0,0,0


## Calculate Number of Yellow Cards Each Team Has and Yellow Card Differential

In [22]:
home_yellows = []
away_yellows = []
delta = []
hy = 0
ay = 0
for index,i in enumerate(event_data['type'].tolist()):
    if 'yellow_card' in i['secondary']:
        if event_data['home_team'].tolist()[index]:
            hy += 1
        else:
            ay += 1
    home_yellows.append(hy)
    away_yellows.append(ay)
    delta.append(hy-ay)
event_data['home_yellow_cards'] = home_yellows
event_data['away_yellow_cards'] = away_yellows
event_data['yellow_card_delta'] = delta
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,end_goal_distance_own,home_goals,away_goals,goal_delta,home_red_cards,away_red_cards,red_card_delta,home_yellow_cards,away_yellow_cards,yellow_card_delta
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,50.000000,0,0,0,0,0,0,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,32.802439,0,0,0,0,0,0,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,53.535035,0,0,0,0,0,0,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,36.124784,0,0,0,0,0,0,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,13.453624,0,0,0,0,0,0,0,0,0


## Calculate Possession Event Index

In [23]:
event_indices = []
for i in event_data['possession'].tolist():
    if i != None:
        event_indices.append(i['eventIndex'] + 1)
    else:
        event_indices.append(0)
event_data['possession_event_index'] = event_indices
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,home_goals,away_goals,goal_delta,home_red_cards,away_red_cards,red_card_delta,home_yellow_cards,away_yellow_cards,yellow_card_delta,possession_event_index
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,0,0,0,0,0,0,0,1
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,0,0,0,0,0,0,0,2
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,0,0,0,0,0,0,0,3
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,0,0,0,0,0,0,0,4
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,0,0,0,0,0,0,0,5


## One Hot Encode Action Height

In [24]:
high = []
nan = []
low = []
blocked = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['pass'] != None:
        height = row['pass']['height']
        if height == 'high':
            high.append(1)
            nan.append(0)
            low.append(0)
            blocked.append(0)
        elif height == 'low':
            high.append(0)
            nan.append(0)
            low.append(1)
            blocked.append(0)
        elif height == 'blocked':
            high.append(0)
            nan.append(0)
            low.append(0)
            blocked.append(1)
        else:
            high.append(0)
            nan.append(1)
            low.append(0)
            blocked.append(0)
    else:
        high.append(0)
        nan.append(1)
        low.append(0)
        blocked.append(0)
event_data['action_height_high'] = high
event_data['action_height_low'] = low
event_data['action_height_blocked'] = blocked
event_data['action_height_nan'] = nan
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,away_red_cards,red_card_delta,home_yellow_cards,away_yellow_cards,yellow_card_delta,possession_event_index,action_height_high,action_height_low,action_height_blocked,action_height_nan
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,0,0,0,1,0,0,0,1
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,0,0,0,2,0,0,0,1
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,0,0,0,3,0,0,0,1
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,0,0,0,4,0,0,0,1
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,0,0,0,5,0,0,0,1


## Calculate Angles

In [25]:
def calc_angle(start_x, start_y, end_x, end_y):
    start_coord = [start_x, start_y]
    end_coord = [end_x, end_y]
    stand_field_length = 1.05
    stand_field_width = 0.68
    # use coor variables to calculate angle between action vector and true forward
    # use goal variables to calculate angle between action vector and vector from action start to the opponent's goal
    x_coor = 100
    y_coor = start_y
    forward_coord = [x_coor, y_coor]

    x_goal = 100
    y_goal = 50

    vec_1 = np.subtract(forward_coord, start_coord)
    vec_2 = np.subtract(end_coord, start_coord)
    
    vec_1[0] = vec_1[0]*stand_field_length
    vec_1[1] = vec_1[1]*stand_field_width
    vec_2[0] = vec_2[0]*stand_field_length
    vec_2[1] = vec_2[1]*stand_field_width

    if np.array_equal(vec_1, [0,0]) or np.array_equal(vec_2, [0,0]):
        angle = 0

    else:
        angle = np.degrees(np.arccos(np.dot(vec_1, vec_2) / (np.sqrt(sum(i**2 for i in vec_1)) * np.sqrt(sum(i**2 for i in vec_2)))))

    if vec_1[0]*vec_2[1] - vec_1[1]*vec_2[0] < 0:
        angle = -angle
        
    return angle

### Angle Between Start and End Location

In [26]:
start_end_angle= []
for index, i in enumerate(event_data['start_x'].tolist()):
    start_x = i
    start_y = event_data['start_y'].tolist()[index]
    end_x = event_data['end_x'].tolist()[index]
    end_y = event_data['end_y'].tolist()[index]
    angle = calc_angle(start_x, start_y, end_x, end_y)
    start_end_angle.append(angle)
event_data['start_end_angle'] = start_end_angle
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,red_card_delta,home_yellow_cards,away_yellow_cards,yellow_card_delta,possession_event_index,action_height_high,action_height_low,action_height_blocked,action_height_nan,start_end_angle
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,0,0,1,0,0,0,1,19.983107
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,0,0,2,0,0,0,1,152.525568
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,0,0,3,0,0,0,1,79.992020
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,0,0,4,0,0,0,1,-112.619865
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,0,0,5,0,0,0,1,-139.398705


### Angle Between Start and Opponent Goal

In [27]:
start_end_angle= []
for index, i in enumerate(event_data['start_x'].tolist()):
    start_x = i
    start_y = event_data['start_y'].tolist()[index]
    end_x = 100
    end_y = 50
    angle = calc_angle(start_x, start_y, end_x, end_y)
    start_end_angle.append(angle)
event_data['start_goal_angle_opponent'] = start_end_angle
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,home_yellow_cards,away_yellow_cards,yellow_card_delta,possession_event_index,action_height_high,action_height_low,action_height_blocked,action_height_nan,start_end_angle,start_goal_angle_opponent
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,0,1,0,0,0,1,19.983107,6.170175
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,0,2,0,0,0,1,152.525568,0.000000
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,0,3,0,0,0,1,79.992020,-9.582945
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,0,4,0,0,0,1,-112.619865,-22.067900
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,0,5,0,0,0,1,-139.398705,-12.835609


### Angle Between Start and Own Goal

In [28]:
start_end_angle= []
for index, i in enumerate(event_data['start_x'].tolist()):
    start_x = i
    start_y = event_data['start_y'].tolist()[index]
    end_x = 0
    end_y = 50
    angle = calc_angle(start_x, start_y, end_x, end_y)
    start_end_angle.append(angle)
event_data['start_goal_angle_own'] = start_end_angle
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,away_yellow_cards,yellow_card_delta,possession_event_index,action_height_high,action_height_low,action_height_blocked,action_height_nan,start_end_angle,start_goal_angle_opponent,start_goal_angle_own
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,1,0,0,0,1,19.983107,6.170175,165.068583
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,2,0,0,0,1,152.525568,0.000000,180.000000
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,3,0,0,0,1,79.992020,-9.582945,-154.290046
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,4,0,0,0,1,-112.619865,-22.067900,-135.000000
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,5,0,0,0,1,-139.398705,-12.835609,-144.246113


### Angle Between End and Opponent Goal

In [29]:
start_end_angle= []
for index, i in enumerate(event_data['end_x'].tolist()):
    start_x = i
    start_y = event_data['end_y'].tolist()[index]
    end_x = 100
    end_y = 50
    angle = calc_angle(start_x, start_y, end_x, end_y)
    start_end_angle.append(angle)
event_data['end_goal_angle_opponent'] = start_end_angle
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,yellow_card_delta,possession_event_index,action_height_high,action_height_low,action_height_blocked,action_height_nan,start_end_angle,start_goal_angle_opponent,start_goal_angle_own,end_goal_angle_opponent
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,1,0,0,0,1,19.983107,6.170175,165.068583,0.000000
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,2,0,0,0,1,152.525568,0.000000,180.000000,-9.582945
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,3,0,0,0,1,79.992020,-9.582945,-154.290046,-22.067900
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,4,0,0,0,1,-112.619865,-22.067900,-135.000000,-12.835609
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,5,0,0,0,1,-139.398705,-12.835609,-144.246113,-3.652223


### Angle Between End and Own Goal

In [30]:
start_end_angle= []
for index, i in enumerate(event_data['end_x'].tolist()):
    start_x = i
    start_y = event_data['end_y'].tolist()[index]
    end_x = 0
    end_y = 50
    angle = calc_angle(start_x, start_y, end_x, end_y)
    start_end_angle.append(angle)
event_data['end_goal_angle_own'] = start_end_angle
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,possession_event_index,action_height_high,action_height_low,action_height_blocked,action_height_nan,start_end_angle,start_goal_angle_opponent,start_goal_angle_own,end_goal_angle_opponent,end_goal_angle_own
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,1,0,0,0,1,19.983107,6.170175,165.068583,0.000000,180.000000
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,2,0,0,0,1,152.525568,0.000000,180.000000,-9.582945,-154.290046
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,3,0,0,0,1,79.992020,-9.582945,-154.290046,-22.067900,-135.000000
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,4,0,0,0,1,-112.619865,-22.067900,-135.000000,-12.835609,-144.246113
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,5,0,0,0,1,-139.398705,-12.835609,-144.246113,-3.652223,-149.036243


## Extract Useful Info From Each Subevent

### Passes

In [31]:
success = []
missing = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['pass'] != None:
        missing.append(0)
        if row['pass']['accurate']:
            success.append(1)
        else:
            success.append(0)
    else:
        success.append(0)
        missing.append(1)
event_data['accurate_pass'] = success
event_data['not_pass_event'] = missing
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,action_height_low,action_height_blocked,action_height_nan,start_end_angle,start_goal_angle_opponent,start_goal_angle_own,end_goal_angle_opponent,end_goal_angle_own,accurate_pass,not_pass_event
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,1,19.983107,6.170175,165.068583,0.000000,180.000000,1,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,1,152.525568,0.000000,180.000000,-9.582945,-154.290046,1,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,1,79.992020,-9.582945,-154.290046,-22.067900,-135.000000,1,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,1,-112.619865,-22.067900,-135.000000,-12.835609,-144.246113,1,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,1,-139.398705,-12.835609,-144.246113,-3.652223,-149.036243,1,0


### Shots

In [32]:
on_target = []
goal = []
missing = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['shot'] != None:
        missing.append(0)
        if row['shot']['onTarget']:
            on_target.append(1)
        else:
            on_target.append(0)
        if row['shot']['isGoal']:
            goal.append(1)
        else:
            goal.append(0)
    else:
        on_target.append(0)
        goal.append(0)
        missing.append(1)
event_data['shot_on_target'] = on_target
event_data['shot_goal'] = goal
event_data['not_shot_event'] = missing
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,start_end_angle,start_goal_angle_opponent,start_goal_angle_own,end_goal_angle_opponent,end_goal_angle_own,accurate_pass,not_pass_event,shot_on_target,shot_goal,not_shot_event
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,19.983107,6.170175,165.068583,0.000000,180.000000,1,0,0,0,1
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,152.525568,0.000000,180.000000,-9.582945,-154.290046,1,0,0,0,1
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,79.992020,-9.582945,-154.290046,-22.067900,-135.000000,1,0,0,0,1
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,-112.619865,-22.067900,-135.000000,-12.835609,-144.246113,1,0,0,0,1
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,-139.398705,-12.835609,-144.246113,-3.652223,-149.036243,1,0,0,0,1


### Ground Duels

In [40]:
missing_off = []
kept_possession = []
progressed_with_ball = []

missing_def = []
stopped_progress = []
recovered_possession = []

takeon = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['groundDuel'] != None:
        if row['groundDuel']['takeOn']:
            takeon.append(1)
        else:
            takeon.append(0)
        if row['groundDuel']['duelType'] == 'defensive_duel':
            missing_def.append(0)
            missing_off.append(1)
            kept_possession.append(0)
            progressed_with_ball.append(0)
            if row['groundDuel']['stoppedProgress']:
                stopped_progress.append(1)
            else:
                stopped_progress.append(0)
            if row['groundDuel']['recoveredPossession']:
                recovered_possession.append(1)
            else:
                recovered_possession.append(0) 
        elif row['groundDuel']['duelType'] == 'offensive_duel' or row['groundDuel']['duelType'] == 'dribble':
            missing_def.append(1)
            missing_off.append(0)
            stopped_progress.append(0)
            recovered_possession.append(0)
            if row['groundDuel']['keptPossession']:
                kept_possession.append(1)
            else:
                kept_possession.append(0)
            if row['groundDuel']['progressedWithBall']:
                progressed_with_ball.append(1)
            else:
                progressed_with_ball.append(0)
        else:
            print(row['groundDuel'])
    else:
        missing_off.append(1)
        missing_def.append(1)
        kept_possession.append(0)
        progressed_with_ball.append(0)
        stopped_progress.append(0)
        recovered_possession.append(0)
        takeon.append(0)
event_data['take_on'] = takeon
event_data['kept_possession_duel'] = kept_possession
event_data['progressed_with_ball_duel'] = progressed_with_ball
event_data['stopped_progress_duel'] = stopped_progress
event_data['recovered_possession_duel'] = recovered_possession
event_data['not_defensive_duel'] = missing_def
event_data['not_offensive_duel'] = missing_off
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,shot_on_target,shot_goal,not_shot_event,take_on,kept_possession_duel,progressed_with_ball_duel,stopped_progress_duel,recovered_possession_duel,not_defensive_duel,not_offensive_duel
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,1,0,0,0,0,0,1,1
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,1,0,0,0,0,0,1,1
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,1,0,0,0,0,0,1,1
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,1,0,0,0,0,0,1,1
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,1,0,0,0,0,0,1,1


### Aerial Duels

In [41]:
first_touch = []
missing = []
height = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['aerialDuel'] != None:
        missing.append(0)
        if row['aerialDuel']['firstTouch']:
            first_touch.append(1)
        else:
            first_touch.append(0)
        height.append(row['aerialDuel']['height'])
    else:
        missing.append(1)
        first_touch.append(0)
        height.append(0)
event_data['first_touch_aerial_duel'] = first_touch
event_data['aerial_duel_height'] = height
event_data['not_aerial_duel'] = missing
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,take_on,kept_possession_duel,progressed_with_ball_duel,stopped_progress_duel,recovered_possession_duel,not_defensive_duel,not_offensive_duel,first_touch_aerial_duel,aerial_duel_height,not_aerial_duel
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,0,0,0,1,1,0,0,1
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,0,0,0,1,1,0,0,1
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,0,0,0,1,1,0,0,1
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,0,0,0,1,1,0,0,1
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,0,0,0,1,1,0,0,1


### Infractions

In [42]:
red = []
yellow = []
missing = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['infraction'] != None:
        missing.append(0)
        if row['infraction']['yellowCard']:
            yellow.append(1)
        else:
            yellow.append(0)
        if row['infraction']['redCard']:
            red.append(1)
        else:
            red.append(0)
    else:
        missing.append(1)
        yellow.append(0)
        red.append(0)
event_data['yellow_card_infraction'] = yellow
event_data['red_card_infraction'] = red
event_data['not_infraction'] = missing
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,stopped_progress_duel,recovered_possession_duel,not_defensive_duel,not_offensive_duel,first_touch_aerial_duel,aerial_duel_height,not_aerial_duel,yellow_card_infraction,red_card_infraction,not_infraction
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,0,1,1,0,0,1,0,0,1
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,0,1,1,0,0,1,0,0,1
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,0,1,1,0,0,1,0,0,1
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,0,1,1,0,0,1,0,0,1
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,0,1,1,0,0,1,0,0,1


### Carries

In [43]:
prog = []
missing = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['carry'] != None:
        missing.append(0)
        prog.append(row['carry']['progression'])
    else:
        missing.append(1)
        prog.append(0)
event_data['progression_carry'] = prog
event_data['not_carry'] = missing
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,not_defensive_duel,not_offensive_duel,first_touch_aerial_duel,aerial_duel_height,not_aerial_duel,yellow_card_infraction,red_card_infraction,not_infraction,progression_carry,not_carry
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,1,1,0,0,1,0,0,1,0.0,1
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,1,1,0,0,1,0,0,1,0.0,1
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,1,1,0,0,1,0,0,1,0.0,1
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,1,1,0,0,1,0,0,1,0.0,1
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,1,1,0,0,1,0,0,1,0.0,1


### Possession

In [44]:
missing = []
trans_low = []
trans_med = []
trans_high = []
attack = []
counter = []
for i in np.arange(len(event_data)):
    row = event_data.iloc[i]
    if row['possession'] != None:
        missing.append(0)
        if row['possession']['types'] == 'transition_low':
            trans_high.append(0)
            trans_low.append(1)
            trans_med.append(0)
            attack.append(0)
            counter.append(0)
        elif row['possession']['types'] == 'transition_medium':
            trans_high.append(0)
            trans_low.append(0)
            trans_med.append(1)
            attack.append(0)
            counter.append(0)
        elif row['possession']['types'] == 'transition_high':
            trans_high.append(1)
            trans_low.append(0)
            trans_med.append(0)
            attack.append(0)
            counter.append(0)
        elif row['possession']['types'] == 'attack':
            trans_high.append(0)
            trans_low.append(0)
            trans_med.append(0)
            attack.append(1)
            counter.append(0)
        elif row['possession']['types'] == 'counterattack':
            trans_high.append(0)
            trans_low.append(0)
            trans_med.append(0)
            attack.append(0)
            counter.append(1)
        else:
            trans_high.append(0)
            trans_low.append(0)
            trans_med.append(0)
            attack.append(0)
            counter.append(0)
    else:
        missing.append(1)
        trans_high.append(0)
        trans_low.append(0)
        trans_med.append(0)
        attack.append(0)
        counter.append(0)
event_data['not_possession'] = missing
event_data['trans_high_possession'] = trans_high
event_data['trans_med_possession'] = trans_med
event_data['trans_low_possession'] = trans_low
event_data['attack_possession'] = attack
event_data['counterattack_possession'] = counter
event_data.head()

,id,matchId,relatedEventId,type,team,opponentTeam,player,pass,shot,groundDuel,...,red_card_infraction,not_infraction,progression_carry,not_carry,not_possession,trans_high_possession,trans_med_possession,trans_low_possession,attack_possession,counterattack_possession
0,1480212789,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684870, 'name': 'K. Groff', 'position':...","{'accurate': True, 'angle': 19, 'height': None...",None,None,...,0,1,0.0,1,0,0,0,0,0,0
1,1480212788,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 655938, 'name': 'M. Cooper', 'position'...","{'accurate': True, 'angle': 152, 'height': Non...",None,None,...,0,1,0.0,1,0,0,0,0,0,0
2,1480212790,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': 80, 'height': None...",None,None,...,0,1,0.0,1,0,0,0,0,0,0
3,1480212791,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['lateral_pas...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 684871, 'name': 'D. Graham', 'position'...","{'accurate': True, 'angle': -111, 'height': No...",None,None,...,0,1,0.0,1,0,0,0,0,0,0
4,1480212792,5415673,1.480213e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'id': 61485, 'name': 'Duke Blue Devils', 'for...","{'id': 61772, 'name': 'Alabama Crimson Tide', ...","{'id': 668789, 'name': 'J. Royson', 'position'...","{'accurate': True, 'angle': -139, 'height': No...",None,None,...,0,1,0.0,1,0,0,0,0,0,0


# Iterate Through all Matches

In [47]:
event_data.columns[0:100]

Index(['id', 'matchId', 'relatedEventId', 'type', 'team', 'opponentTeam',
       'player', 'pass', 'shot', 'groundDuel', 'aerialDuel', 'infraction',
       'carry', 'possession', 'match_time_seconds', '1H', '2H', '1E', '2E',
       'home_team', 'pass_primary_dummy', 'interception_primary_dummy',
       'corner_primary_dummy', 'goalkeeper_exit_primary_dummy',
       'shot_primary_dummy', 'goal_kick_primary_dummy', 'duel_primary_dummy',
       'throw_in_primary_dummy', 'touch_primary_dummy',
       'infraction_primary_dummy', 'free_kick_primary_dummy',
       'clearance_primary_dummy', 'shot_against_primary_dummy',
       'acceleration_primary_dummy', 'offside_primary_dummy',
       'back_pass_secondary_dummy', 'short_or_medium_pass_secondary_dummy',
       'lateral_pass_secondary_dummy', 'linkup_play_secondary_dummy',
       'under_pressure_secondary_dummy', 'forward_pass_secondary_dummy',
       'long_pass_secondary_dummy', 'pass_to_final_third_secondary_dummy',
       'progressive_pas

In [42]:
#CODE THAT APPENDS EVENT DATA INTO A BIG DF
big_event_data = pd.DataFrame()
j = 1
for i in tqdm(matches_df['matchId'].tolist()):
    event_data = requests.get(f"https://apirest.wyscout.com/v3/matches/{i}/events", auth=(username, password))
    try:
        data = pd.DataFrame(event_data.json()['events'])
    except:
        print(f"Error {j}")
        j = j + 1
        continue
    big_event_data = pd.concat([big_event_data, data])


  0%|          | 1/5314 [00:04<6:25:25,  4.35s/it]

Error 1



  0%|          | 7/5314 [00:17<3:52:14,  2.63s/it]

Error 2



  0%|          | 8/5314 [00:18<3:02:31,  2.06s/it]

...

Error 94



 22%|██▏       | 1153/5314 [1:55:40<13:53:53, 12.02s/it]

Error 95



 22%|██▏       | 1167/5314 [1:58:25<17:03:58, 14.82s/it]

MemoryError: Unable to allocate 207. MiB for an array with shape (15, 1806912) and data type object

In [44]:
matches_df['matchId'].tolist()[0]
event_data = requests.get(f"https://apirest.wyscout.com/v3/matches/5539787/events", auth=(username, password))
data = pd.DataFrame(event_data.json()['events'])
data.head()

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,type,location,team,opponentTeam,player,pass,shot,groundDuel,aerialDuel,infraction,carry,possession
0,1800671870,5539787,1H,0,4,00:00:04.561,6.561177,1.800672e+09,"{'primary': 'pass', 'secondary': ['back_pass',...","{'x': 50, 'y': 51}","{'id': 61422, 'name': 'Bryant Bulldogs', 'form...","{'id': 61833, 'name': 'Chicago State Cougars',...","{'id': 758799, 'name': 'J. Collins', 'position...","{'accurate': True, 'angle': -138, 'height': No...",None,None,None,None,None,"{'id': 1800671870, 'duration': '56.713308', 't..."
1,1800671871,5539787,1H,0,7,00:00:07.122,9.122475,1.800672e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 42, 'y': 39}","{'id': 61422, 'name': 'Bryant Bulldogs', 'form...","{'id': 61833, 'name': 'Chicago State Cougars',...","{'id': 711160, 'name': 'M. Griffin', 'position...","{'accurate': True, 'angle': -45, 'height': Non...",None,None,None,None,None,"{'id': 1800671870, 'duration': '56.713308', 't..."
2,1800671872,5539787,1H,0,9,00:00:09.097,11.097184,1.800672e+09,"{'primary': 'pass', 'secondary': ['forward_pas...","{'x': 59, 'y': 13}","{'id': 61422, 'name': 'Bryant Bulldogs', 'form...","{'id': 61833, 'name': 'Chicago State Cougars',...","{'id': 711161, 'name': 'N. Singer', 'position'...","{'accurate': True, 'angle': 0, 'height': None,...",None,None,None,None,None,"{'id': 1800671870, 'duration': '56.713308', 't..."
3,1800671873,5539787,1H,0,9,00:00:09.357,11.357735,1.800672e+09,"{'primary': 'duel', 'secondary': ['ground_duel...","{'x': 67, 'y': 13}","{'id': 61422, 'name': 'Bryant Bulldogs', 'form...","{'id': 61833, 'name': 'Chicago State Cougars',...","{'id': 758759, 'name': 'C. Bean', 'position': ...",None,None,"{'opponent': {'id': 760662, 'name': 'B. Sanche...",None,None,None,"{'id': 1800671870, 'duration': '56.713308', 't..."
4,1800673166,5539787,1H,0,9,00:00:09.357,11.357735,1.800672e+09,"{'primary': 'duel', 'secondary': ['defensive_d...","{'x': 33, 'y': 87}","{'id': 61833, 'name': 'Chicago State Cougars',...","{'id': 61422, 'name': 'Bryant Bulldogs', 'form...","{'id': 760662, 'name': 'B. Sanchez', 'position...",None,None,"{'opponent': {'id': 758759, 'name': 'C. Bean',...",None,None,None,"{'id': 1800671870, 'duration': '56.713308', 't..."
